In [80]:
import os 
os.environ["TFF_CPP_MIN_LOG_LEVEL"]="2"

In [81]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [82]:
# physical_device=tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_device[0],True)

In [83]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

In [84]:
print(ds_info)

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\klikh\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [85]:
# fig = tfds.show_examples(ds_train,ds_info,rows=4,cols=4)

when we want to show the image we need to set as_supervised=False.

In [86]:

def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [87]:
AUTOTUNE=tf.data.experimental.AUTOTUNE
BATCH_SIZE=128

In [88]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [89]:
# Setup for test Dataset
# ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
# ds_test = ds_train.batch(128)
# ds_test = ds_train.prefetch(AUTOTUNE)
ds_test=ds_test.map(normalize_img,num_parallel_calls=AUTOTUNE).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [90]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)


In [91]:
model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [92]:
model.fit(ds_train, epochs=5, verbose=2)

Epoch 1/5


469/469 - 14s - loss: 0.2645 - accuracy: 0.9261 - 14s/epoch - 30ms/step
Epoch 2/5
469/469 - 10s - loss: 0.0957 - accuracy: 0.9732 - 10s/epoch - 22ms/step
Epoch 3/5
469/469 - 10s - loss: 0.0668 - accuracy: 0.9808 - 10s/epoch - 22ms/step
Epoch 4/5
469/469 - 10s - loss: 0.0510 - accuracy: 0.9849 - 10s/epoch - 22ms/step
Epoch 5/5
469/469 - 10s - loss: 0.0415 - accuracy: 0.9875 - 10s/epoch - 22ms/step


In [93]:
model.evaluate(ds_test,verbose=2)

79/79 - 1s - loss: 0.0586 - accuracy: 0.9801 - 1s/epoch - 14ms/step


[0.05863616243004799, 0.9800999760627747]